In [1]:
import pandas as pd
import numpy as np
import math
import scipy
from scipy.stats import norm
from dataclasses import dataclass
pd.options.mode.chained_assignment = None

### **Trade quantities**
* Sell $N_p = 1000$ put contracts
* Buy $N_c$ call contracts (paid for using put premium)
* Cash leftover: $C_0$ from strategy

### **Underlying**
* SPY price:
  * At initiation: $S_0$
  * One week before expiry: $S_t$

### **Option values**
* Put:
  * At initiation: $V_{p,0}$
  * At time $t$: $V_{p,t}$
* Call:
  * At initiation: $V_{c,0}$
  * At time $t$: $V_{c,t}$

---

## **Premium-Financing Constraint**

5 % OTM Put-sale premium finances 5 % OTM calls plus leftover cash:

$$1000 \cdot 100 \cdot V_{p,0} = N_c \cdot 100 \cdot V_{c,0} + C_0$$

Hence:

$$N_c = \frac{1000 \cdot 100 \cdot V_{p,0} - C_0}{V_{c,0} \cdot 100}$$

---

## **P&L One Week Before Expiry**

Total mark-to-market P&L at time $t$ (A single options contract for SPY represents 100 shares of the SPDR S&P 500 ETF Trust):

$$\text{PNL}_t = \underbrace{1000 \cdot  100 \cdot (V_{p,0} - V_{p,t})}_{\text{Short Put P\&L}} + \underbrace{N_c \cdot 100 \cdot (V_{c,t} - V_{c,0})}_{\text{Long Call P\&L}} + \underbrace{C_0(e^{rt} - 1)}_{\text{Interest on Cash}}$$

Percentage Return with respect to put premium collected

$$\text{Percentage Return on Premium Collected (\%)} = \frac{\text{PNL}_t \cdot 100}{\text{Put Sale Premium at Time 0}}$$

---

## **Interpretation**

* If the 95% put decays or SPY rises → **positive** put P&L
* If the 105% call gains value → **positive** call P&L
* Cash earns risk-free interest at rate $r$ over time $t$
* Combined structure ≈ **short vol + long convexity financed using premium + interest on residual cash**

In [2]:
class OptionsPricingModel:
    """Options pricing model for SPY options pricing"""
    
    def __init__(self, spot_price, risk_free_rate, dividend_yield):
        self.S0 = spot_price
        self.r = risk_free_rate
        self.q = dividend_yield  # SPY dividend yield
        
    def black_scholes_price(self, K, T, sigma, option_type='call'):
        """
        Calculate Black-Scholes price for European options
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Implied volatility
        option_type: 'call' or 'put'
        """
        d1 = (np.log(self.S0 / K) + (self.r - self.q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type == 'call':
            price = self.S0 * np.exp(-self.q * T) * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)
        else:  # put
            price = K * np.exp(-self.r * T) * norm.cdf(-d2) - self.S0 * np.exp(-self.q * T) * norm.cdf(-d1)
        
        return price
    
    
    def crr_binomial_price(self, K, T, sigma, option_type='call', steps=1000, american=True):
        """
        Calculate option price using Cox-Ross-Rubinstein binomial tree
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Volatility
        option_type: 'call' or 'put'
        steps: Number of time steps in the binomial tree
        american: True for American options, False for European
        """
        dt = T / steps  # Time increment
        
        # CRR parameters
        u = np.exp(sigma * np.sqrt(dt))  # Up factor
        d = 1 / u  # Down factor
        p = (np.exp((self.r - self.q) * dt) - d) / (u - d)  # Risk-neutral probability
        discount = np.exp(-self.r * dt)  # Discount factor per step
        
        # Initialize asset prices at maturity
        ST = np.zeros(steps + 1)
        for i in range(steps + 1):
            ST[i] = self.S0 * (u ** (steps - i)) * (d ** i)
        
        # Initialize option values at maturity
        if option_type == 'call':
            option_values = np.maximum(ST - K, 0)
        else:  # put
            option_values = np.maximum(K - ST, 0)
        
        # Backward induction through the tree
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                # Current stock price at this node
                S = self.S0 * (u ** (step - i)) * (d ** i)
                
                # Continuation value (discounted expected value)
                continuation_value = discount * (p * option_values[i] + (1 - p) * option_values[i + 1])
                
                # Early exercise value
                if option_type == 'call':
                    exercise_value = max(S - K, 0)
                else:  # put
                    exercise_value = max(K - S, 0)
                
                # For American options, take the maximum
                if american:
                    option_values[i] = max(continuation_value, exercise_value)
                else:
                    option_values[i] = continuation_value
        
        return option_values[0]

In [3]:
@dataclass
class TradeInputs:
    # Market / model inputs
    spot_price: float
    risk_free_rate: float
    dividend_yield: float
    time_to_maturity: float  # in years

    # Put (SELL) 95% moneyness
    put_strike: float
    put_contracts: int
    put_iv: float
    put_market_price: float

    # Call (BUY) 105% moneyness
    call_strike: float
    call_iv: float
    call_market_price: float

    # Binomial settings
    binomial_steps: int
    american: bool

class BullishRiskReversalInitialSetUpAnalyzer:
    """
    Encapsulates the full pricing + report logic
    for the short put / long call trade.
    """

    def __init__(self, inputs: TradeInputs):
        self.inputs = inputs
        # Initialize the underlying pricing model
        self.model = OptionsPricingModel(
            spot_price=inputs.spot_price,
            risk_free_rate=inputs.risk_free_rate,
            dividend_yield=inputs.dividend_yield,
        )

    def compute(self) -> dict:
        """
        Run all pricing steps and return a dictionary of results
        """
        i = self.inputs

        # === Step 4: Price the options (Black–Scholes) ===
        put_price_bs = self.model.black_scholes_price(
            K=i.put_strike,
            T=i.time_to_maturity,
            sigma=i.put_iv,
            option_type='put'
        )

        call_price_bs = self.model.black_scholes_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call'
        )

        # === Step 4b: Price the options (CRR Binomial) ===
        put_price_binom = self.model.crr_binomial_price(
            K=i.put_strike,
            T=i.time_to_maturity,
            sigma=i.put_iv,
            option_type='put',
            steps=i.binomial_steps,
            american=i.american
        )

        call_price_binom = self.model.crr_binomial_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call',
            steps=i.binomial_steps,
            american=i.american
        )

        # === Step 5: Trade economics (using the current market price for trading) ===

        # Premium collected from selling puts
        premium_collected = i.put_market_price * i.put_contracts * 100

        # Max number of call contracts that can afforded with that premium
        call_contracts = int(premium_collected // (i.call_market_price * 100))

        # Premium paid for calls using that size
        premium_paid = i.call_market_price * call_contracts * 100

        net_premium = premium_collected - premium_paid

        return {
            "spot_price": i.spot_price,
            "risk_free_rate": i.risk_free_rate,
            "dividend_yield": i.dividend_yield,
            "time_to_maturity": i.time_to_maturity,
            "binomial_steps": i.binomial_steps,

            "put_strike": i.put_strike,
            "put_iv": i.put_iv,
            "put_contracts": i.put_contracts,
            "put_market_price": i.put_market_price,
            "put_price_bs": put_price_bs,
            "put_price_binom": put_price_binom,

            "call_strike": i.call_strike,
            "call_iv": i.call_iv,
            "call_market_price": i.call_market_price,
            "call_price_bs": call_price_bs,
            "call_price_binom": call_price_binom,

            # Trade economics based on the market price at T0
            "premium_collected": premium_collected,
            "call_contracts": call_contracts,
            "premium_paid": premium_paid,
            "net_premium": net_premium,
        }

    def print_report(self) -> None:
        """
        Pretty-print the same info the script printed,
        using the results from compute().
        """
        r = self.compute()

        print("=== TRADE SETUP ===")
        print(f"Spot Price: ${r['spot_price']:.2f}")
        print(f"Risk-free rate: {r['risk_free_rate']*100:.2f}%")
        print(f"Dividend yield: {r['dividend_yield']*100:.2f}%")
        print(f"Time to maturity: {r['time_to_maturity']:.4f} years")
        print(f"Binomial steps: {r['binomial_steps']}")

        print("\n--- PUT (SELL) 95% Moneyness ---")
        print(f"Strike: ${r['put_strike']:.2f}")
        print(f"Implied Vol: {r['put_iv']*100:.2f}%")
        print(f"Contracts: {r['put_contracts']}")
        print(f"  Market price per Put option:       ${r['put_market_price']:.4f}")
        print(f"  BS price per option:       ${r['put_price_bs']:.4f}")
        print(f"  Binomial price per option: ${r['put_price_binom']:.4f}")
        print(f"  Premium collected (market): ${r['premium_collected']:,.2f}")

        print("\n--- CALL (BUY) 105% Moneyness ---")
        print(f"Strike: ${r['call_strike']:.2f}")
        print(f"Implied Vol: {r['call_iv']*100:.2f}%")
        print(f"Call Contracts bought: {r['call_contracts']}")
        print(f"  Market price per call option:       ${r['call_market_price']:.4f}")
        print(f"  BS price per option:       ${r['call_price_bs']:.4f}")
        print(f"  Binomial price per option: ${r['call_price_binom']:.4f}")
        print(f"  Premium paid (market): ${r['premium_paid']:,.2f}")

        print("\n=== NET PREMIUM (based on market prices) ===")
        print(f"Net Premium: ${r['net_premium']:,.2f}")

        print("\n=== MODEL CHECK ===")
        print(
            "Put:  BS vs Binomial =",
            round(r['put_price_bs'], 4), "vs", round(r['put_price_binom'], 4)
        )
        print(
            "Call: BS vs Binomial =",
            round(r['call_price_bs'], 4), "vs", round(r['call_price_binom'], 4)
        )


In [4]:
def compute_trade_pnl(
    n_put_contracts: int,
    n_call_contracts: int,
    V_p_0: float,
    V_p_t: float,
    V_c_0: float,
    V_c_t: float,
    net_premium: float,        # C0
    risk_free_rate: float,     # r
    rf_compounding_period: float    # t in years
) -> tuple:
    """
    Compute the PNL of the risk reversal,

    Formula used:
        PNL_t =
            + n_put_contracts  * 100 * (V_p_0 - V_p_t)
            + n_call_contracts * 100 * (V_c_t - V_c_0)
            + net_premium * (e^(r*t) - 1)

    Where:
        n_put_contracts  : number of short put option contracts
        n_call_contracts : number of long call option contracts
        V_p_0, V_p_t     : initial and final put prices
        V_c_0, V_c_t     : initial and final call prices
        net_premium      : net premium (cash flow at t=0)
        risk_free_rate   : r
        rf_compounding_period : t in years

    Returns: pnl_total : Total Pnl of the strategy
    """

    # --- PNL from short puts ---
    term_put = n_put_contracts * 100 * (V_p_0 - V_p_t)

    # --- PNL from long calls ---
    term_call = n_call_contracts * 100 * (V_c_t - V_c_0)

    # --- Carry on cash premium ---
    term_carry = net_premium * (math.exp(risk_free_rate * rf_compounding_period) - 1)

    pnl_total = term_put + term_call + term_carry

    premium_collected = n_put_contracts * 100 * V_p_0
    pnl_perecntage = pnl_total / premium_collected

    return pnl_total, pnl_perecntage



def risk_reversal_greeks(
    n_put,
    n_call,
    put_delta,
    call_delta,
    put_gamma,
    call_gamma,
    put_vega,
    call_vega,
    put_theta,
    call_theta,
    put_rho,
    call_rho,
    contract_size=100,
):
    """
    Compute portfolio Greeks for a risk reversal.

    n_put, n_call : number of contracts (positive = long, negative = short)
    Greeks are per-option (not per-contract).
    contract_size : usually 100 for US equity options.
    """

    # Each Greek = sum(contracts * contract_size * per-option greek)
    delta = contract_size * (n_call * call_delta + n_put * put_delta)
    gamma = contract_size * (n_call * call_gamma + n_put * put_gamma)
    vega  = contract_size * (n_call * call_vega  + n_put * put_vega)
    theta = contract_size * (n_call * call_theta + n_put * put_theta)
    rho   = contract_size * (n_call * call_rho   + n_put * put_rho)

    return {
        "delta": delta,
        "gamma": gamma,
        "vega": vega,
        "theta": theta,
        "rho": rho,
    }


In [5]:
# Load 5% OTM option data for the given date and define the rf and dividend rate
five_percent_otm_options_path = '../data/Option/eod_option_5%_OTM_2025-04-09.csv'
df = pd.read_csv(five_percent_otm_options_path)
risk_free_rate=0.0433
dividend_yield=0.0127
timet_to_maturity = 30/365

# Separate calls and puts

call = df[df["call_put"] == "C"]
put  = df[df["call_put"] == "P"]

# Spot price is identical for all rows; take the first
spot_price = call["underlying_price"].iloc[0]


# Extract Greeks and key fields for the 5% OTM call
call_market_price = call["price"].iloc[0]
call_iv     = call["iv"].iloc[0]
call_strike = call["price_strike"].iloc[0]
call_delta  = call["delta"].iloc[0]
call_gamma  = call["gamma"].iloc[0]
call_vega   = call["vega"].iloc[0]
call_theta  = call["theta"].iloc[0]
call_rho    = call["rho"].iloc[0]


# Extract Greeks and key fields for the 5% OTM put
put_market_price = put["price"].iloc[0]
put_iv      = put["iv"].iloc[0]
put_strike  = put["price_strike"].iloc[0]
put_delta   = put["delta"].iloc[0]
put_gamma   = put["gamma"].iloc[0]
put_vega    = put["vega"].iloc[0]
put_theta   = put["theta"].iloc[0]
put_rho     = put["rho"].iloc[0]

# ------------------Print results---------------
print(f"Spot Price: {spot_price}")

print("\n--- CALL (5% OTM) ---")
print(f"Call Market Price: {call_market_price}")
print(f"IV:     {call_iv}")
print(f"Strike: {call_strike}")
print(f"Delta:  {call_delta}")
print(f"Gamma:  {call_gamma}")
print(f"Vega:   {call_vega}")
print(f"Theta:  {call_theta}")
print(f"Rho:    {call_rho}")

print("\n--- PUT (5% OTM) ---")
print(f"Put Market Price: {put_market_price}")
print(f"IV:     {put_iv}")
print(f"Strike: {put_strike}")
print(f"Delta:  {put_delta}")
print(f"Gamma:  {put_gamma}")
print(f"Vega:   {put_vega}")
print(f"Theta:  {put_theta}")
print(f"Rho:    {put_rho}")


Spot Price: 543.37

--- CALL (5% OTM) ---
Call Market Price: 4.225
IV:     0.210945
Strike: 571.0
Delta:  0.228211
Gamma:  0.009197
Vega:   0.470783
Theta:  -0.176452
Rho:    0.098448

--- PUT (5% OTM) ---
Put Market Price: 8.475
IV:     0.323284
Strike: 516.0
Delta:  -0.264147
Gamma:  0.006544
Vega:   0.51151
Theta:  -0.262369
Rho:    -0.111529


In [6]:
inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=risk_free_rate,
    dividend_yield=dividend_yield,
    time_to_maturity=timet_to_maturity ,

    put_strike=put_strike,
    put_contracts=1000,
    put_iv=put_iv,
    put_market_price=put_market_price,

    call_strike=call_strike,
    call_iv=call_iv,
    call_market_price=call_market_price,

    binomial_steps=1000,
    american=True,
)

analyzer = BullishRiskReversalInitialSetUpAnalyzer(inputs=inputs)
analyzer.print_report()
initial_set_up = analyzer.compute()

initial_set_up

=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.33%
Dividend yield: 1.27%
Time to maturity: 0.0822 years
Binomial steps: 1000

--- PUT (SELL) 95% Moneyness ---
Strike: $516.00
Implied Vol: 32.33%
Contracts: 1000
  Market price per Put option:       $8.4750
  BS price per option:       $8.4602
  Binomial price per option: $8.4911
  Premium collected (market): $847,500.00

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Call Contracts bought: 2005
  Market price per call option:       $4.2250
  BS price per option:       $4.1938
  Binomial price per option: $4.1914
  Premium paid (market): $847,112.50

=== NET PREMIUM (based on market prices) ===
Net Premium: $387.50

=== MODEL CHECK ===
Put:  BS vs Binomial = 8.4602 vs 8.4911
Call: BS vs Binomial = 4.1938 vs 4.1914


{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.0433,
 'dividend_yield': 0.0127,
 'time_to_maturity': 0.0821917808219178,
 'binomial_steps': 1000,
 'put_strike': np.float64(516.0),
 'put_iv': np.float64(0.323284),
 'put_contracts': 1000,
 'put_market_price': np.float64(8.475),
 'put_price_bs': np.float64(8.460236074876917),
 'put_price_binom': np.float64(8.491084290464878),
 'call_strike': np.float64(571.0),
 'call_iv': np.float64(0.210945),
 'call_market_price': np.float64(4.225),
 'call_price_bs': np.float64(4.193847130448077),
 'call_price_binom': np.float64(4.191387904323273),
 'premium_collected': np.float64(847500.0),
 'call_contracts': 2005,
 'premium_paid': np.float64(847112.5),
 'net_premium': np.float64(387.5)}

# **Validation of Options Pricing Model Against Market Data**

We will validate our pricing model using the 1-month SPY options chain from April 9, 2025. To avoid low-liquidity distortions, we restrict the analysis to options with moneyness between 80% and 120%, since far ITM Call & OTM Put contracts often trade infrequently and therefore exhibit unreliable market prices.

Using a risk-free rate of 4.33% (from the 1-month Treasury yield) and a dividend yield of 1.27% based on the most recent distribution, we input these parameters into both the Black–Scholes and Binomial models and compare their theoretical values against the mid-market prices of the listed options.

## 1. Per-option percentage error

Let
$$
P_i^{\text{mkt}} = \text{market mid price}, \qquad
P_i^{\text{model}} = \text{model price}.
$$

Define the **percentage error**:

$$
e_i = \frac{P_i^{\text{model}} - P_i^{\text{mkt}}}{P_i^{\text{mkt}}}.
$$


---

## 2. Aggregate metrics across all options

### **Mean Absolute Percentage Error (MAPE)**

$$
\text{MAPE} =
\frac{100}{N}\sum_{i=1}^{N}
\left|
\frac{P_i^{\text{model}} - P_i^{\text{mkt}}}{P_i^{\text{mkt}}}
\right|.
$$

This provides how far on average the model deviates from the mid price in percent(%) terms.

### **Root Mean Squared Percentage Error (RMSPE)**

$$
\text{RMSPE} =
100 \sqrt{
\frac{1}{N} \sum_{i=1}^{N}
\left(
\frac{P_i^{\text{model}} - P_i^{\text{mkt}}}{P_i^{\text{mkt}}}
\right)^{2}
}.
$$

Same idea as above put penalizes larger errors more.


## 3. Mean Percentage Error (MPE)

Signed mean percentage error:

$$
\text{MPE} = \frac{100}{N} \sum_{i=1}^{N} \frac{P_i^{\text{model}} - P_i^{\text{mkt}}}{P_i^{\text{mkt}}}
$$

Positive → model overpriced on average; negative → underpriced.

In [7]:
df_eod_options_chain = pd.read_csv('../data/Option/eod_option_SPY_2025-04-09.csv')
df_eod_options_chain_filtered = df_eod_options_chain[(df_eod_options_chain['calc_OTM'] <= 20) & (df_eod_options_chain['calc_OTM'] >=-20)]
df_eod_options_chain_filtered

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,...,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,is_settlement
54,2025-04-09,SPY 250509C00435000,30,627,2025-05-09,C,435.0,NaN,NaN,NaN,...,0.001439,-0.198121,0.185609,0.327572,112.20,111.31,543.37,-19.94,137973657,0
55,2025-04-09,SPY 250509P00435000,30,627,2025-05-09,P,435.0,8.72,9.06,1.68,...,0.001384,-0.145575,0.173022,-0.026148,2.03,1.93,543.37,19.94,137973658,0
56,2025-04-09,SPY 250509C00440000,30,627,2025-05-09,C,440.0,65.51,65.51,65.51,...,0.001572,-0.204801,0.197696,0.329274,107.36,106.51,543.37,-19.02,137973659,0
57,2025-04-09,SPY 250509P00440000,30,627,2025-05-09,P,440.0,9.65,10.12,1.78,...,0.001517,-0.151607,0.188355,-0.026790,2.19,2.07,543.37,19.02,137973660,0
58,2025-04-09,SPY 250509C00445000,30,627,2025-05-09,C,445.0,61.38,61.38,61.38,...,0.001706,-0.208041,0.207716,0.331365,102.47,101.64,543.37,-18.10,137973661,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2025-04-09,SPY 250509P00640000,30,627,2025-05-09,P,640.0,106.00,110.00,106.00,...,0.000000,0.000000,0.000000,0.000000,97.95,94.64,543.37,-17.78,137973714,0
276,2025-04-09,SPY 250509C00645000,30,627,2025-05-09,C,645.0,0.03,0.03,0.03,...,0.000558,-0.013216,0.032307,0.003281,0.11,0.07,543.37,18.70,137973715,0
277,2025-04-09,SPY 250509P00645000,30,627,2025-05-09,P,645.0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,102.95,99.64,543.37,-18.70,137973716,0
278,2025-04-09,SPY 250509C00650000,30,627,2025-05-09,C,650.0,0.04,0.10,0.04,...,0.000421,-0.009971,0.024398,0.002391,0.06,0.03,543.37,19.62,137973717,0


In [8]:
time_to_maturity = 30 / 365

pricing_model = OptionsPricingModel(
    spot_price=543.37,
    risk_free_rate=0.0423,
    dividend_yield=0.0127
)

for index, row in df_eod_options_chain_filtered.iterrows():
    strike = row['price_strike']
    iv = row['iv']

    if row['call_put'] == 'C':
        bs_price = pricing_model.black_scholes_price(
            strike, time_to_maturity, iv, 'call'
        )
        binomial_price = pricing_model.crr_binomial_price(
            strike, time_to_maturity, iv, 'call', steps=1000, american=True
        )

    elif row['call_put'] == 'P':
        bs_price = pricing_model.black_scholes_price(
            strike, time_to_maturity, iv, 'put'
        )
        binomial_price = pricing_model.crr_binomial_price(
            strike, time_to_maturity, iv, 'put', steps=1000, american=True
        )

    # write back to the DataFrame
    df_eod_options_chain_filtered.loc[index, 'bs_price'] = bs_price
    df_eod_options_chain_filtered.loc[index, 'binomial_price'] = binomial_price


In [9]:
df = df_eod_options_chain_filtered

# Percentage errors
df['pct_bs_error']  = (df['bs_price'] - df['price']) / df['price']
df['pct_bin_error'] = (df['binomial_price'] - df['price']) / df['price']

# Black-Scholes
bs_mape  = df['pct_bs_error'].abs().mean() * 100
bs_rmspe = (df['pct_bs_error']**2).mean()**0.5 * 100
bs_mpe   = df['pct_bs_error'].mean() * 100

# Binomial Tree
bin_mape  = df['pct_bin_error'].abs().mean() * 100
bin_rmspe = (df['pct_bin_error']**2).mean()**0.5 * 100
bin_mpe   = df['pct_bin_error'].mean() * 100


print("=== MODEL ERROR METRICS (%) ===\n")

print("Black-Scholes Model:")
print(f"  MAPE : {bs_mape:.4f}%")
print(f"  RMSPE: {bs_rmspe:.4f}%")
print(f"  MPE  : {bs_mpe:.4f}%\n")

print("Binomial Model:")
print(f"  MAPE : {bin_mape:.4f}%")
print(f"  RMSPE: {bin_rmspe:.4f}%")
print(f"  MPE  : {bin_mpe:.4f}%")


=== MODEL ERROR METRICS (%) ===

Black-Scholes Model:
  MAPE : 0.8345%
  RMSPE: 2.8362%
  MPE  : -0.2678%

Binomial Model:
  MAPE : 0.7723%
  RMSPE: 2.7728%
  MPE  : 0.0976%


### **Model Performance Summary**

Both the Black–Scholes and the Binomial pricing models perform well using the chosen parameters, each achieving a **MAPE below 1%**, indicating a close fit between model prices and observed market mid-prices. However, the **Binomial model outperforms Black–Scholes across all three accuracy metrics** (MAPE, RMSPE, and MPE).

Because the Binomial model consistently yields lower error and also correctly accommodates the possibility of **early exercise in American-style options**, it is the more appropriate choice for further analysis.

In [10]:
risk_reversal_greeks(
    initial_set_up['put_contracts'],
    initial_set_up['call_contracts'],
    put_delta,
    call_delta,
    put_gamma,
    call_gamma,
    put_vega,
    call_vega,
    put_theta,
    call_theta,
    put_rho,
    call_rho,
    contract_size=100,
)

{'delta': np.float64(19341.605499999998),
 'gamma': np.float64(2498.3985000000002),
 'vega': np.float64(145542.9915),
 'theta': np.float64(-61615.526),
 'rho': np.float64(8585.923999999999)}

### **Key Observations from Portfolio Greeks**

Following the historic rally, the trader initiated a bullish risk-reversal position—short 95% puts and long 105% calls—and the resulting Greeks reveal an aggressively leveraged, high-convexity bullish profile that carries significant risk.

**Delta: +19,341.61**
The portfolio exhibits an extremely strong positive delta, implying substantial bullish exposure. This level of delta suggests that the long call position, funded by premium collected from the short puts, is very large (~2000 contracts). As a result, the portfolio gains roughly **$19,300** for every $1 increase in SPY. The trader is essentially holding a large synthetic long equity position.

**Gamma: +2,498.40**
The positive gamma indicates that the portfolio’s delta will increase further as SPY rises, providing upside acceleration. This convexity is characteristic of being net long options, meaning the long calls overwhelmingly dominate the profile. However, on the downside, although gamma reduces the delta as SPY falls, the trader still carries substantial exposure through the large short put position.

**Vega: +145,542.99**
The portfolio is massively long volatility. A 1-point drop in implied volatility results in approximately **$145,500** of losses. Entering the trade after a major rally is particularly dangerous, as implied volatility typically collapses following such events. This creates a scenario where the trader is exposed to significant mark-to-market losses even if the underlying moves upward.

**Theta: –61,615.53**
Time decay is severe: the trader loses about **$61,600 per day** from theta alone. With only a month until expiration, this decay will accelerate, requiring the underlying to move sharply and quickly to offset the decay.

**Rho: +8,585.92**
The position has moderately positive rho, benefiting from rising interest rates. While this is far less impactful than the delta, vega, and theta exposures, it still contributes marginally to the overall profile.

---

### **Critical Risks Identified**

1. **Volatility Crush Risk**
   Because the position is so heavily long volatility, any post-rally volatility collapse could cause substantial losses. This is especially concerning given the historical tendency for implied volatility to compress sharply after large upward moves.

2. **Severe Theta Bleed**
   The portfolio is losing more than $62,000 per day due to time decay. Given the short time to expiration, the trader faces rapidly accelerating theta losses unless SPY rallies hard and fast.

3. **Timing Risk**
   The trader entered after a massive rally traditionally the worst time to purchase upside calls:

   * Calls are inflated due to demand and elevated skew dynamics.
   * Puts have already seen their fear premium eroded.
   * Markets often consolidate or retrace after major rallies.

4. **Asymmetric Downside Risk**
   While the upside is theoretically unlimited, the short puts introduce substantial downside risk. Any sharp reversal could lead to significant losses well before the calls regain value.

---

### **Conclusion**

Overall, the trader has constructed an aggressively bullish position at a moment when the risk–reward profile is unfavorable. The extreme vega and theta exposure means that without a **rapid and continued upside breakout**, the position is vulnerable to large mark-to-market losses. Even a benign sideways market or a modest volatility contraction could turn the trade sharply negative.


# **Scenario Analysis for 1-mo SPY 95% Put Short & 105% Call Long**

### **Trade Structure (as of 4/9/2025 afternoon)**

* **Sell 1000× 1-month 95% moneyness SPY puts**

  * Initial IV (5% OTM put): **0.323**
* **Buy 1000× 1-month 105% moneyness SPY calls (using the premium)**

  * Initial IV (5% OTM call): **0.211**

### **Objective**

Project **1-week-before-expiry P&L**, under:

1. **Spot moves (over 3 weeks):**
   +15%, +10%, +5%, +2.5%, 0%, −2.5%, −5%, −10% -15%
2. **Implied volatility adjustments:**
   Need multiple reasonable IV paths for both the **OTM call** and **OTM put**.

---

# **IV Change Assumptions**

A 3-week move toward expiry typically produces:

* **Volatility–spot correlation:** For equities, **negative**.

  * Spot up → IVs fall (especially downside puts).
  * Spot down → IVs rise (especially downside puts).
* **Skew dynamics:**

  * Downside IV rises *more* than upside IV.
  * Upside IV is relatively stable unless there is a large rally.
* **Time-to-expiry shrink:**

  * Skew becomes steeper near expiry because crash-risk premiums concentrate in short tenors.

Because of this, IV changes can be represented in **three regimes**:

1. **Spot Up → IV Softening**
2. **Spot Flat → Slight Normalization**
3. **Spot Down → IV Expansion (skew steepening)**

---

# **Scenario Grid (IV % Changes)**

All IV changes are relative to the initial IVs:

* **Call IV₀ = 21.1%**
* **Put  IV₀ = 32.3%**

Values are expressed as **percentage change of implied volatility**, not absolute points.

---


| Spot Change (SPY) | Scenario       | ΔIV (105% Call) | ΔIV (95% Put) | Short Market Reasoning                                                                                      |
| ----------------- | -------------- | --------------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| **+15%**          | vol_collapse   | −9%             | −25%          | Violent melt-up; crash premium dumped, skew sharply flattens as downside hedges are unwound.                |
|                   | normal_decline | −6%             | −17%          | Strong but orderly rally; systematic vol sellers and lack of demand for puts drive broad vol roll-down.     |
|                   | mixed_regime   | −3%             | −10%          | Rally continues but some macro uncertainty remains; skew compresses, but downside still holds residual bid. |
|                   | persistent_vol | 0%              | −4%           | Price rips higher but realized vol stays elevated; calls hold bid while downside IV slowly normalizes.      |
| **+10%**          | vol_collapse   | −7%             | −18%          | Continued risk-on; short-gamma covering and structured vol supply crush downside tails.                     |
|                   | normal_decline | −5%             | −13%          | Healthy trend up; hedging demand fades and both wings drift lower, more so in puts.                         |
|                   | mixed_regime   | −2%             | −7%           | Grind higher with pockets of macro risk; skew eases but investors keep some tail protection.                |
|                   | persistent_vol | +2%             | −2%           | Rally but with elevated realized swings; upside optionality bid while downside slowly cheapens.             |
| **+5%**           | vol_collapse   | −5%             | −12%          | Relief rally after prior stress; vol sellers lean in and downside IV compresses meaningfully.               |
|                   | normal_decline | −3%             | −7%           | Moderate grind up; steady reduction in put demand and skew.                                                 |
|                   | mixed_regime   | 0%              | 0%            | Quiet drift higher; term-structure roll-down offset by mild macro worries, leaving surface unchanged.       |
|                   | persistent_vol | +3%             | +3%           | Rally occurs in choppy tape; both wings retain a premium due to higher realized vol-of-vol.                 |
| **+2.5%**         | vol_collapse   | −2%             | −5%           | Small extension of risk-on move; dealers supply vol and downside IV softens.                                |
|                   | normal_decline | −1%             | −2%           | Mild upside; theta decay plus roll-down gradually cheapen both call and put IVs.                            |
|                   | mixed_regime   | +2%             | +4%           | Spot drifts up but data risk looms; investors pay for short-dated gamma on both sides.                      |
|                   | persistent_vol | +5%             | +7%           | Choppy, headline-driven rise; volatility of volatility stays high and both wings remain well bid.           |
| **0%**            | vol_collapse   | −2%             | −2%           | Market trades sideways with low realized vol; surface mean-reverts slightly lower.                          |
|                   | normal_decline | 0%              | 0%            | Balanced flows; theta and roll-down largely priced in, surface effectively flat.                            |
|                   | mixed_regime   | +3%             | +5%           | Range-bound but event risk ahead; gamma and skew see a modest bid.                                          |
|                   | persistent_vol | +6%             | +10%          | Sticky uncertainty; elevated hedging and gamma-trading keep both wings rich, especially downside.           |
| **−2.5%**         | vol_collapse   | 0%              | +2%           | Small pullback after strong run; modest put demand, but no real fear.                                       |
|                   | normal_decline | +1%             | +4%           | Dip is bought but hedgers add some protection; skew steepens a bit.                                         |
|                   | mixed_regime   | +3%             | +9%           | Repeated minor selloffs; investors start to chase downside protection and skew.                             |
|                   | persistent_vol | +5%             | +13%          | Choppy down-tape; systematic hedging flows lift both wings with a stronger bid for puts.                    |
| **−5%**           | vol_collapse   | +2%             | +5%           | Initial flush, then stabilization; downside IV marks higher but not aggressively as dip buyers step in.     |
|                   | normal_decline | +3%             | +9%           | Classic risk-off day; hedging demand increases and skew moves steeper.                                      |
|                   | mixed_regime   | +5%             | +14%          | Repeated tests of lower levels; dealers and real money both pay up for crash protection.                    |
|                   | persistent_vol | +8%             | +20%          | Sustained volatile drawdown; vol-of-vol high, downside IV re-prices toward stress levels.                   |
| **−10%**          | vol_collapse   | +2%             | +6%           | Large move but largely anticipated (e.g., event); vol rises, yet less than historical panic norms.          |
|                   | normal_decline | +5%             | +16%          | Strong hedging flows after a sharp selloff; puts command sizeable crash premium.                            |
|                   | mixed_regime   | +8%             | +24%          | Fear re-emerges; vol buyers dominate and skew steepens materially.                                          |
|                   | persistent_vol | +12%            | +30%          | Extended risk-off environment; downside vol moves into clear stress-regime territory.                       |
| **−15%**          | vol_collapse   | +3%             | +8%           | Deep correction but with earlier hedges in place; vol rises but remains below full-blown panic.             |
|                   | normal_decline | +7%             | +22%          | Crisis-style hedging; crash-risk premium jumps and skew becomes very steep.                                 |
|                   | mixed_regime   | +11%            | +32%          | High-stress environment; repeated gaps and liquidity concerns drive heavy demand for puts.                  |
|                   | persistent_vol | +16%            | +40%          | Near-panic regime (VIX mid-40s); forced selling and crash hedging send downside IV to crisis levels.        |

In [11]:
iv_scenarios = {
    0.15: {
        "vol_collapse":   {"call_iv_change": -0.09, "put_iv_change": -0.25},  # Strong melt-up, big vol crush
        "normal_decline": {"call_iv_change": -0.06, "put_iv_change": -0.17},  # Steady normalization
        "mixed_regime":   {"call_iv_change": -0.03, "put_iv_change": -0.10},  # Gradual stabilization
        "persistent_vol": {"call_iv_change":  0.00, "put_iv_change": -0.04}   # Calls flat, puts still decay
    },
    0.10: {
        "vol_collapse":   {"call_iv_change": -0.07, "put_iv_change": -0.18},  # Continued rally confidence
        "normal_decline": {"call_iv_change": -0.05, "put_iv_change": -0.13},  # Steady stabilization
        "mixed_regime":   {"call_iv_change": -0.02, "put_iv_change": -0.07},  # Modest normalization
        "persistent_vol": {"call_iv_change":  0.02, "put_iv_change": -0.02}   # Slight call bid, put decay
    },
    0.05: {
        "vol_collapse":   {"call_iv_change": -0.05, "put_iv_change": -0.12},  # Continued vol compression
        "normal_decline": {"call_iv_change": -0.03, "put_iv_change": -0.07},  # Mild normalization
        "mixed_regime":   {"call_iv_change":  0.00, "put_iv_change":  0.00},  # Term structure decay offsets
        "persistent_vol": {"call_iv_change":  0.03, "put_iv_change":  0.03}   # Modest both-way increase
    },
    0.025: {
        "vol_collapse":   {"call_iv_change": -0.02, "put_iv_change": -0.05},  # Time decay dominates
        "normal_decline": {"call_iv_change": -0.01, "put_iv_change": -0.02},  # Minimal change
        "mixed_regime":   {"call_iv_change":  0.02, "put_iv_change":  0.04},  # Slight uncertainty premium
        "persistent_vol": {"call_iv_change":  0.05, "put_iv_change":  0.07}   # Both pick up modestly
    },
    0.0: {
        "vol_collapse":   {"call_iv_change": -0.02, "put_iv_change": -0.02},  # Pure time decay / mild mean reversion
        "normal_decline": {"call_iv_change":  0.00, "put_iv_change":  0.00},  # Flat
        "mixed_regime":   {"call_iv_change":  0.03, "put_iv_change":  0.05},  # Range uncertainty
        "persistent_vol": {"call_iv_change":  0.06, "put_iv_change":  0.10}   # Elevated gamma hedging
    },
    -0.025: {
        "vol_collapse":   {"call_iv_change":  0.00, "put_iv_change":  0.02},  # Minimal reaction
        "normal_decline": {"call_iv_change":  0.01, "put_iv_change":  0.04},  # Slight defensive bid
        "mixed_regime":   {"call_iv_change":  0.03, "put_iv_change":  0.09},  # Growing concern
        "persistent_vol": {"call_iv_change":  0.05, "put_iv_change":  0.13}   # Vol starting to pick up
    },
    -0.05: {
        "vol_collapse":   {"call_iv_change":  0.02, "put_iv_change":  0.05},  # Modest both-way increase
        "normal_decline": {"call_iv_change":  0.03, "put_iv_change":  0.09},  # Normal hedging flow
        "mixed_regime":   {"call_iv_change":  0.05, "put_iv_change":  0.14},  # Defensive positioning
        "persistent_vol": {"call_iv_change":  0.08, "put_iv_change":  0.20}   # Clear vol regime shift
    },
    -0.10: {
        "vol_collapse":   {"call_iv_change":  0.02, "put_iv_change":  0.06},  # Muted vol rise despite drawdown
        "normal_decline": {"call_iv_change":  0.05, "put_iv_change":  0.16},  # Strong hedging demand
        "mixed_regime":   {"call_iv_change":  0.08, "put_iv_change":  0.24},  # Fear re-emerging
        "persistent_vol": {"call_iv_change":  0.12, "put_iv_change":  0.30}   # Elevated stress
    },
    -0.15: {
        "vol_collapse":   {"call_iv_change":  0.03, "put_iv_change":  0.08},  # Vol rises, but not panic
        "normal_decline": {"call_iv_change":  0.07, "put_iv_change":  0.22},  # Crisis hedging
        "mixed_regime":   {"call_iv_change":  0.11, "put_iv_change":  0.32},  # High stress environment
        "persistent_vol": {"call_iv_change":  0.16, "put_iv_change":  0.40}   # Near-panic (VIX mid-40s)
    }
}

inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=risk_free_rate,
    dividend_yield=dividend_yield,
    time_to_maturity=time_to_maturity,

    put_strike=put_strike,
    put_contracts=1000,
    put_iv=put_iv,
    put_market_price=put_market_price,

    call_strike=call_strike,
    call_iv=call_iv,
    call_market_price=call_market_price,

    binomial_steps=1000,
    american=True,
)

analyzer = BullishRiskReversalInitialSetUpAnalyzer(inputs=inputs)
analyzer.print_report()      
initial_set_up = analyzer.compute()

initial_set_up 


=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.33%
Dividend yield: 1.27%
Time to maturity: 0.0822 years
Binomial steps: 1000

--- PUT (SELL) 95% Moneyness ---
Strike: $516.00
Implied Vol: 32.33%
Contracts: 1000
  Market price per Put option:       $8.4750
  BS price per option:       $8.4602
  Binomial price per option: $8.4911
  Premium collected (market): $847,500.00

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Call Contracts bought: 2005
  Market price per call option:       $4.2250
  BS price per option:       $4.1938
  Binomial price per option: $4.1914
  Premium paid (market): $847,112.50

=== NET PREMIUM (based on market prices) ===
Net Premium: $387.50

=== MODEL CHECK ===
Put:  BS vs Binomial = 8.4602 vs 8.4911
Call: BS vs Binomial = 4.1938 vs 4.1914


{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.0433,
 'dividend_yield': 0.0127,
 'time_to_maturity': 0.0821917808219178,
 'binomial_steps': 1000,
 'put_strike': np.float64(516.0),
 'put_iv': np.float64(0.323284),
 'put_contracts': 1000,
 'put_market_price': np.float64(8.475),
 'put_price_bs': np.float64(8.460236074876917),
 'put_price_binom': np.float64(8.491084290464878),
 'call_strike': np.float64(571.0),
 'call_iv': np.float64(0.210945),
 'call_market_price': np.float64(4.225),
 'call_price_bs': np.float64(4.193847130448077),
 'call_price_binom': np.float64(4.191387904323273),
 'premium_collected': np.float64(847500.0),
 'call_contracts': 2005,
 'premium_paid': np.float64(847112.5),
 'net_premium': np.float64(387.5)}

In [12]:
def scenario_simulation(
    initial_set_up: dict,
    iv_scenarios: dict,
    model: type[OptionsPricingModel],
    pricing_model_used: str,
    time_left_to_expiry: float,
    rf_compounding_period: float
) -> dict:
    """
    Run spot + IV scenarios and return, for each (spot_move, scenario_name):
        - V_p_t: scenario put price
        - V_c_t: scenario call price
        - pnl_dollars: PnL of the strategy in dollars
        - pnl_pct: PnL as a percentage of |net_premium|
    """

    # -------------------------------------------- Unpack initial setup at T0 ----------------------------------------------
    initial_spot_price = initial_set_up["spot_price"]
    n_put_contracts    = initial_set_up["put_contracts"]
    n_call_contracts   = initial_set_up["call_contracts"]
    put_strike         = initial_set_up["put_strike"]
    call_strike        = initial_set_up["call_strike"]
    put_iv             = initial_set_up["put_iv"]
    call_iv            = initial_set_up["call_iv"]
    net_premium        = initial_set_up["net_premium"]
    risk_free_rate     = initial_set_up["risk_free_rate"]
    dividend_yield     = initial_set_up["dividend_yield"]
    binomial_steps     = initial_set_up["binomial_steps"]

    V_p_0 = initial_set_up["put_market_price"]
    V_c_0 = initial_set_up["call_market_price"]

    # -------------------------------------------- Calculate Pnl for each scenario ----------------------------------------------

    # --- Container for results ---
    scenario_simulation_result: dict = {}

    # --- Loop over spot and IV scenarios ---
    for spot_move, scenario_dict in iv_scenarios.items():
        # Ensure nested dict exists
        scenario_simulation_result[spot_move*100] = {}

        # New spot is a percentage move from initial
        new_spot = initial_spot_price * (1 + spot_move)

        for scenario_name, iv_changes in scenario_dict.items():
            call_change = iv_changes["call_iv_change"]
            put_change  = iv_changes["put_iv_change"]

            # Apply IV changes
            new_put_iv  = put_iv  * (1 + put_change)
            new_call_iv = call_iv * (1 + call_change)


            print("\n--- SPOT MOVE ---")
            print(f"Before spot: {initial_spot_price}")
            print(f"Change : {spot_move * 100}%")
            print(f"After  spot: {new_spot}")

            print("\n--- IMPLIED VOLATILITY CHANGES ---")
            print("PUT IV:")
            print(f"  Before: {put_iv}")
            print(f"  Change: {put_change * 100:+}%")
            print(f"  After : {new_put_iv}")

            print("\nCALL IV:")
            print(f"  Before: {call_iv}")
            print(f"  Change: {call_change * 100:+}%")
            print(f"  After : {new_call_iv}")


            # Build a pricing model at the new spot
            pricing_model = model(
                spot_price=new_spot,
                risk_free_rate=risk_free_rate,
                dividend_yield=dividend_yield,
            )

            # Price options under chosen model
            if pricing_model_used == "bs":
                V_p_t = pricing_model.black_scholes_price(
                    K=put_strike,
                    T=time_left_to_expiry,
                    sigma=new_put_iv,
                    option_type="put",
                )
                V_c_t = pricing_model.black_scholes_price(
                    K=call_strike,
                    T=time_left_to_expiry,
                    sigma=new_call_iv,
                    option_type="call",
                )

            elif pricing_model_used == "binomial":
                V_p_t = pricing_model.crr_binomial_price(
                    K=put_strike,
                    T=time_left_to_expiry,
                    sigma=new_put_iv,
                    option_type="put",
                    steps=binomial_steps,
                    american=True,
                )
                V_c_t = pricing_model.crr_binomial_price(
                    K=call_strike,
                    T=time_left_to_expiry,
                    sigma=new_call_iv,
                    option_type="call",
                    steps=binomial_steps,
                    american=True,
                )
            else:
                raise ValueError(f"Unknown pricing_model_used: {pricing_model_used}")

            # Compute PnL of the strategy
            pnl, pnl_percentage = compute_trade_pnl(
                n_put_contracts=n_put_contracts,
                n_call_contracts=n_call_contracts,
                V_p_0=V_p_0,
                V_p_t=V_p_t,
                V_c_0=V_c_0,
                V_c_t=V_c_t,
                net_premium=net_premium,
                risk_free_rate=risk_free_rate,
                rf_compounding_period=rf_compounding_period,
            )

            scenario_simulation_result[spot_move*100][scenario_name] = {
                "S_0": initial_spot_price, 
                "S_t": round(new_spot, 2),

                "put_iv_0":  round(put_iv,4),
                "Change_in_put_iv (%)":  round((new_put_iv / put_iv - 1) * 100, 4) if put_iv  != 0 else None,
                "put_iv_t":  round(new_put_iv,4),

                "call_iv_0": round(call_iv,4),
                "Change_in_call_iv (%)": round((new_call_iv / call_iv - 1) * 100, 4) if call_iv != 0 else None,
                "call_iv_t": round(new_call_iv,4),

                "V_p_0": round(V_p_0,4),
                "Change_in_put_price (%)":  round((V_p_t / V_p_0 - 1) * 100,4) if V_p_0 != 0 else None,
                "V_p_t": round(V_p_t,4),

                "V_c_0": round(V_c_0,4),
                "Change_in_call_price (%)": round((V_c_t / V_c_0 - 1) * 100,4) if V_c_0 != 0 else None,
                "V_c_t": round(V_c_t,4),

                "pnl_dollars": round(pnl,2),
                "pnl_percentage": round(pnl_percentage,2)
            }


    return scenario_simulation_result

def simulation_result_to_df(scenario_simulation_result):
    """
    Convert a nested dictionary of the form:
        { move: { scenario: { field: value, ... } } }
    into a MultiIndex pandas DataFrame.
    
    Index levels:
        level 0 → move
        level 1 → scenario
    """
    
    # Flatten nested structure into tuple keys
    flat_dict = {
        (move, scenario): metrics
        for move, scenarios in scenario_simulation_result.items()
        for scenario, metrics in scenarios.items()
    }

    # Build DataFrame
    df = pd.DataFrame.from_dict(flat_dict, orient="index")

    # Set MultiIndex with names
    df.index = pd.MultiIndex.from_tuples(df.index, names=["Spot Move(%)", "scenario"])

    return df


In [13]:
scenario_simulation_result = scenario_simulation(
    initial_set_up,
    iv_scenarios,
    OptionsPricingModel,
    pricing_model_used ='binomial',
    time_left_to_expiry=7/365,
    rf_compounding_period=23/365
)

scenario_simulation_result 


--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -25.0%
  After : 0.242463

CALL IV:
  Before: 0.210945
  Change: -9.0%
  After : 0.19195995

--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -17.0%
  After : 0.26832572

CALL IV:
  Before: 0.210945
  Change: -6.0%
  After : 0.19828829999999997

--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -10.0%
  After : 0.29095560000000004

CALL IV:
  Before: 0.210945
  Change: -3.0%
  After : 0.20461664999999998

--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES ---
PUT IV:
  Before: 0.323284
  Change: -4.0%
  After : 0.31035264

CALL IV:
  Before: 0.210945
  Change: +0.0%
  After : 0.210945

--- S

{15.0: {'vol_collapse': {'S_0': np.float64(543.37),
   'S_t': np.float64(624.88),
   'put_iv_0': np.float64(0.3233),
   'Change_in_put_iv (%)': np.float64(-25.0),
   'put_iv_t': np.float64(0.2425),
   'call_iv_0': np.float64(0.2109),
   'Change_in_call_iv (%)': np.float64(-9.0),
   'call_iv_t': np.float64(0.192),
   'V_p_0': np.float64(8.475),
   'Change_in_put_price (%)': np.float64(-100.0),
   'V_p_t': np.float64(0.0),
   'V_c_0': np.float64(4.225),
   'Change_in_call_price (%)': np.float64(1182.8066),
   'V_c_t': np.float64(54.1986),
   'pnl_dollars': np.float64(10867203.99),
   'pnl_percentage': np.float64(12.82)},
  'normal_decline': {'S_0': np.float64(543.37),
   'S_t': np.float64(624.88),
   'put_iv_0': np.float64(0.3233),
   'Change_in_put_iv (%)': np.float64(-17.0),
   'put_iv_t': np.float64(0.2683),
   'call_iv_0': np.float64(0.2109),
   'Change_in_call_iv (%)': np.float64(-6.0),
   'call_iv_t': np.float64(0.1983),
   'V_p_0': np.float64(8.475),
   'Change_in_put_price (%)': 

In [14]:
df_scenario_analysis_result = simulation_result_to_df(scenario_simulation_result)

df_scenario_analysis_result['avg_pnl_for_spot_move'] = (
    df_scenario_analysis_result
    .groupby('Spot Move(%)')['pnl_dollars']
    .transform('mean').map('{:,.2f}'.format)
)

df_scenario_analysis_result.to_csv('../data/Result/1_Scenario_Analysis_Result.csv')
df_scenario_analysis_result

S_0     S_t  put_iv_0  Change_in_put_iv (%)  \
Spot Move(%) scenario                                                         
 15.0        vol_collapse    543.37  624.88    0.3233                 -25.0   
             normal_decline  543.37  624.88    0.3233                 -17.0   
             mixed_regime    543.37  624.88    0.3233                 -10.0   
             persistent_vol  543.37  624.88    0.3233                  -4.0   
 10.0        vol_collapse    543.37  597.71    0.3233                 -18.0   
             normal_decline  543.37  597.71    0.3233                 -13.0   
             mixed_regime    543.37  597.71    0.3233                  -7.0   
             persistent_vol  543.37  597.71    0.3233                  -2.0   
 5.0         vol_collapse    543.37  570.54    0.3233                 -12.0   
             normal_decline  543.37  570.54    0.3233                  -7.0   
             mixed_regime    543.37  570.54    0.3233                   0.0   
             persistent_vol  543.37  570.54    0.3233                   3.0   
 2.5         vol_collapse    543.37  556.95    0.3233                  -5.0   
             normal_decline  543.37  556.95    0.3233                  -2.0   
             mixed_regime    543.37  556.95    0.3233                   4.0   
             persistent_vol  543.37  556.95    0.3233                   7.0   
 0.0         vol_collapse    543.37  543.37    0.3233                  -2.0   
             normal_decline  543.37  543.37    0.3233                   0.0   
             mixed_regime    543.37  543.37    0.3233                   5.0   
             persistent_vol  543.37  543.37    0.3233                  10.0   
-2.5         vol_collapse    543.37  529.79    0.3233                   2.0   
             normal_decline  543.37  529.79    0.3233                   4.0   
             mixed_regime    543.37  529.79    0.3233                   9.0   
             persistent_vol  543.37  529.79    0.3233                  13.0   
-5.0         vol_collapse    543.37  516.20    0.3233                   5.0   
             normal_decline  543.37  516.20    0.3233                   9.0   
             mixed_regime    543.37  516.20    0.3233                  14.0   
             persistent_vol  543.37  516.20    0.3233                  20.0   
-10.0        vol_collapse    543.37  489.03    0.3233                   6.0   
             normal_decline  543.37  489.03    0.3233                  16.0   
             mixed_regime    543.37  489.03    0.3233                  24.0   
             persistent_vol  543.37  489.03    0.3233                  30.0   
-15.0        vol_collapse    543.37  461.86    0.3233                   8.0   
             normal_decline  543.37  461.86    0.3233                  22.0   
             mixed_regime    543.37  461.86    0.3233                  32.0   
             persistent_vol  543.37  461.86    0.3233                  40.0   

                             put_iv_t  call_iv_0  Change_in_call_iv (%)  \
Spot Move(%) scenario                                                     
 15.0        vol_collapse      0.2425     0.2109                   -9.0   
             normal_decline    0.2683     0.2109                   -6.0   
             mixed_regime      0.2910     0.2109                   -3.0   
             persistent_vol    0.3104     0.2109                    0.0   
 10.0        vol_collapse      0.2651     0.2109                   -7.0   
             normal_decline    0.2813     0.2109                   -5.0   
             mixed_regime      0.3007     0.2109                   -2.0   
             persistent_vol    0.3168     0.2109                    2.0   
 5.0         vol_collapse      0.2845     0.2109                   -5.0   
             normal_decline    0.3007     0.2109                   -3.0   
             mixed_regime      0.3233     0.2109                    0.0   
             persistent_vol    0.3330     0.2109      